In [1]:
!pip install mesa-geo

In [2]:
!gdown --id "1ou6VXPnVG-icFiK-3vNwiwDmmnju46tw"

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1ou6VXPnVG-icFiK-3vNwiwDmmnju46tw
To: /content/new-york-city-boroughs.geojson
100% 2.67M/2.67M [00:00<00:00, 133MB/s]


In [3]:
!pip install pyngrok

In [4]:
!ngrok config add-authtoken 2WBVyOjBuQYHAe5T0JdShzDTcsW_2dxZVTUVJ6CeG3qfUApsc

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [5]:
import mesa_geo as mg
import geopandas as gpd
import matplotlib.pyplot as plt
import mesa

In [6]:
from shapely.geometry import Point

#for data collector
def infected_count(model):
    return (sum(1 for agent in model.schedule.agents if agent.atype == "infected"))

def susceptible_count(model):
    return (sum(1 for agent in model.schedule.agents if agent.atype == "susceptible"))

def dead_count(model):
    return (sum(1 for agent in model.schedule.agents if agent.atype == "dead"))

class Person(mg.GeoAgent):
    def __init__(self, unique_id, model,geometry,crs,agent_type="susceptible",range=150,recovery_rate=0.2,death_risk=0.1,init_infected=0.1,):
        super().__init__(unique_id, model, geometry, crs)
        self.atype = agent_type
        self.range = range
        self.recovery_rate = recovery_rate
        self.death_risk = death_risk
        #chance of person starting as infected
        if self.random.random() < init_infected:
            self.atype = "infected"

    def step(self):
        self.check_status()
        self.move()

    def check_status(self):
        #infects
        if self.atype == "infected":
            if self.random.random() < self.recovery_rate:
                self.atype = "susceptible"
            elif self.random.random() < self.death_risk:
                self.atype = "dead"
        if self.atype == "infected":
            neighbors = self.model.space.get_neighbors_within_distance(self, self.model.exposure_distance)
            for neighbor in neighbors:
                if (neighbor.atype == "susceptible" and self.random.random() < self.model.infection_risk):
                    neighbor.atype = "infected"
                    break

    def move(self):

        if self.atype != "dead":
            move_x = self.random.randint(-self.range, self.range)
            move_y = self.random.randint(-self.range, self.range)
            self.geometry = Point(self.geometry.x + move_x, self.geometry.y + move_y)
        '''
        #freezes model whenever it is run like this. my theory is that some agents are too close to the edge so it takes a very long time to generate a viable coordinate
        if self.atype != "dead":
            while True:
                move_x = self.random.randint(-self.range, self.range)
                move_y = self.random.randint(-self.range, self.range)
                newLoc = Point(self.geometry.x + move_x, self.geometry.y + move_y)
                neighbors = [agent for agent in self.model.space.get_intersecting_agents(newLoc) if isinstance(agent, BoroughAgent)]
                if neighbors:
                    self.geometry = newLoc
                    break
        '''
    def __repr__(self):
        return "Person " + str(self.unique_id)


class BoroughAgent(mg.GeoAgent):
    def __init__(self, unique_id, model, geometry, crs, agent_type="safe", hotspot_threshold=5):
        super().__init__(unique_id, model, geometry, crs)
        self.atype = agent_type
        self.hotspot_threshold = (
            hotspot_threshold
        )
        self.detect_hotspot()

    def step(self):
        self.detect_hotspot()

    def detect_hotspot(self):
        # Region is hotspot if more agents infected within than above threshold
        neighbors = self.model.space.get_intersecting_agents(self)
        infected_neighbors = [
            neighbor for neighbor in neighbors if neighbor.atype == "infected"
        ]
        if len(infected_neighbors) <= self.hotspot_threshold:
            self.atype = "safe"
        else:
            self.atype = "hotspot"

    def __repr__(self):
        return "Neighborhood " + str(self.unique_id)




In [7]:
class GeoInf(mesa.Model):
    # Geographical parameters for desired map
    geojson_regions = "new-york-city-boroughs.geojson"
    unique_id = "name"

    def __init__(
        self, n=30, percInf=0.2, expDist=500, infRisk=0.2
    ):
        self.schedule = mesa.time.BaseScheduler(self)
        self.space = mg.GeoSpace(warn_crs_conversion = False)

        # SIR model parameters
        self.pop = n
        self.exposure_distance = expDist
        self.infection_risk = infRisk

        self.running = True
        self.datacollector = mesa.DataCollector(
            {
                "infected": infected_count,
                "susceptible": susceptible_count,
                "dead": dead_count,
            }
        )
      # Create Borough agents
        ac = mg.AgentCreator(BoroughAgent, model=self)
        borough_agents = ac.from_file(
            self.geojson_regions, unique_id=self.unique_id
        )
        self.space.add_agents(borough_agents)

        # Create persons
        ac_population = mg.AgentCreator(
            Person,
            model=self,
            crs=self.space.crs,
            agent_kwargs={"init_infected": percInf},
        )
        # Add person to random borough in random location
        for i in range(n):
            #keep trying until agent is within a borough
            while True:
                borough = self.random.randint(0, len(borough_agents) - 1)
                center_x, center_y = borough_agents[borough].geometry.centroid.coords.xy
                bounds = borough_agents[borough].geometry.bounds
                spread_x = int(bounds[2] - bounds[0])
                spread_y = int(bounds[3] - bounds[1])

                x = center_x[0] + self.random.randint(0, spread_x) - spread_x / 2
                y = center_y[0] + self.random.randint(0, spread_y) - spread_y / 2
                new_person = ac_population.create_agent(Point(x, y), "P" + str(i))

                if borough_agents[borough].geometry.intersects(new_person.geometry):
                    self.space.add_agents(new_person)
                    self.schedule.add(new_person)
                    break

        for agent in borough_agents:
            self.schedule.add(agent)
        self.datacollector.collect(self)


    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)

        # Run until no one is infected
        if infected_count(self) == 0:
            self.running = False

In [8]:
model_params = {
    "n": mesa.visualization.Slider("Population size", 100, 10, 200, 10),
    "percInf": mesa.visualization.Slider(
        "Percent infected", 0.5, 0.00, 1.0, 0.05),
    "expDist": mesa.visualization.Slider(
        "Exposure distance", 600, 100, 1000, 100),
}


def portrayAgents(agent):
    portrayal = {}
    if isinstance(agent, Person):
        portrayal["radius"] = "2"
    if agent.atype in ["hotspot", "infected"]:
        portrayal["color"] = "Red"
    elif agent.atype in ["safe", "susceptible"]:
        portrayal["color"] = "Green"
    elif agent.atype in ["dead"]:
        portrayal["color"] = "Black"
    return portrayal

map = mg.visualization.MapModule(portrayAgents)
chart = mesa.visualization.ChartModule(
    [
        {"Label": "infected", "Color": "Red"},
        {"Label": "susceptible", "Color": "Green"},
        {"Label": "dead", "Color": "Black"},
    ]
)
server = mesa.visualization.ModularServer(
    GeoInf,
    [map, chart],
    "Basic Geo Infection model",
    model_params,
)


In [ ]:
from pyngrok import ngrok
#pyngrok sets up tunnel to local server on web
#need because google collab is a virtual machine so cant' automatically open tab to display local servers content like mesa visualization server intends.
url = ngrok.connect(8521)
print('Open the following URL in a new tab to access the server:', url)
server.launch()